# Inspecting the Python Parser for Quirk
As discussed in class, each of our implementation languages makes one of either the lexer, parser or interpreter difficult. For Python, the parser is difficult as building a parser tree is one of a class of algorithms that is challenging to create iteratively while being easier when decomposed recursively (or functionally). As such, you should think of each subtree as its own problem. Also, each of these problems addresses a single grammar rule (e.g. `<Program> -> <Statement> <Program> | <Statement>`). 

The reference code for this Jupyter Notebook is from [PartialParser.py](https://github.com/dr-jam/csc521/blob/master/course-materials/PartialParser.py).

## Basics
### Token Stream
The parser takes a token stream as a input. In the full version of your Quirk implementations, the parser will begin by reading the full set of tokens (including and EOF token at the very end of the stream) via standard input and store them in a global array named tokens.
For testing purposes, you can declare a list of tokens to test with. An example would be:
```python
tokens = ["VAR", "IDENT:X", "ASSIGN", "NUMBER:4", "EOF"]
```
You can see more examples at the top of [PartialParser.py](https://github.com/dr-jam/csc521/blob/master/course-materials/PartialParser.py).

### Token Index
Each of the grammar functions has a single parameter called `token_index` which contains the position in the `token` list where the grammar should begin parsing from. This is not kept globally like `token` as grammar functions can fail. Keeping the relevant index locally in each function makes backtracking on grammar failure substantially easier.

### Return Values of Grammar Functions

The return values of each of these grammar functions is a list of three values:
1. A boolean value that is Ture if a subtree that corresponds to the grammar found and False if not.
2. The position into the list of tokens where the grammar function left off.
3. The parse tree generated by the grammar function.

Here is an example of using multiple variable assignemnt to pull out each of thse values in the return value list into their own local variable names:
```python
(success, returned_index, returned_subtree) = Statement(token_index)
```

Here is an example the return values of a call to the `Expression` grammar function where it successfuly parsed `-x`:
```Python
[True,
 2,
 ['Expression2', ['Term2', ['Factor4', ['Value0', ['Name1', 'SUB', 'IDENT:X']]]]]
]
```
with:
```python
tokens = ["SUB", "IDENT:x", "EOF"]
```

Unsuccessful parses return `False`, the `token_index` passed in to the grammar function that failed (after all, failure shouldn't consume tokens as we want to use them again when after backtracking), and an empty subtree:
```python
return [False, token_index, []]
```

### Numbers After Nonterminals
Note the numbers after the names of the nonterminals in the parse tree:
```Python
['Expression2', 
    ['Term2', 
        ['Factor4', 
            ['Value0', 
                ['Name1', 'SUB', 'IDENT:X']
            ]
        ]
    ]
]

```
<a id="factors"></a>Those numbers correspond to the replacement rule that the parser used to generated that part of the parse tree. These numbers are useful to the interpreter and are quite handy when debugging your parser. Take the the `<Factor>` portion of the grammar as an example:
```
<Factor> -> <SubExpression> EXP <Factor> | <SubExpression> | <FunctionCall> 
    | <Value> EXP <Factor> | <Value>
```
It contains fives ways to replace replace the nonterminal `<Factor>` (each of which is followed by its name in the parse tree) :
0. `<SubExpression> EXP <Factor>` (Factor0)
1. `<SubExpression>` (Factor1)
2. `<FunctionCall>` (Factor2)
3. `<Value> EXP <Factor>` (Factor3)
4. `<Value>` (Factor4)

There should be a replacement rule number after every nonterminal in your parse tree include those like `<Print>` that only have a single replacement rule.

### Handling Token:Lexeme Pairs
Utility functions that determine if a token is a NAME or NUMBER are provided for you:

In [1]:
def is_ident(tok):
    '''Determines if the token is of type IDENT.
    tok - a token
    returns True if IDENT is in the token or False if not.
    '''
    return -1 < tok.find("IDENT")

def is_number(tok):
    '''Determines if the token is of type NUMBER.
    tok - a token
    returns True if NUMBER is in the token or False if not.
    '''
    return -1 < tok.find("NUMBER")

## First Grammar Function: Number
Consider the following part of the parser that was given to you in PartialParser.py:

In [2]:
def Number(token_index):
    '''<Number> ->
        NUMBER
        | SUB NUMBER
        | ADD NUMBER
    '''
    if is_number(tokens[token_index]):
        subtree = ["Number0", tokens[token_index]]
        return [True, token_index + 1, subtree]
    
    if "SUB" == tokens[token_index]:
        if is_number(tokens[token_index + 1]):
            subtree = ["Number1", tokens[token_index], tokens[token_index + 1]]
            return [True, token_index + 2, subtree]
        
    if "ADD" == tokens[token_index]:
        if is_number(tokens[token_index + 1]):
            subtree = ["Number2", tokens[token_index], tokens[token_index + 1]]
            return [True, token_index + 2, subtree]
        
    return [False, token_index, []]

This is one of the base cases that stops the recursive construction of the parse tree. You can tell that it is a base case because by looking at the replacement rules for <Number> and seeing that thay are all terminals (i.e. tokens). This function (along with it's counterpart the `Name` grammar function) creates leaves in the parse tree.

It contains four sections of code. The first three start with `if` statements and detect the token streams for the `Number0`, `Number1`, and `Number2` replacement rules. When they are successful, they build appropriate parse tree leaves for their cases. Note that the second and 3rd sections provide support for negative and positive numbers.

Let's run the grammar function on a stream of tokens:

In [3]:
tokens = ["SUB", "NUMBER:1.2", "EOF"]
Number(0) #Starts parsing <Number> at the first token in the list

[True, 2, ['Number1', 'SUB', 'NUMBER:1.2']]

By inspecting the return values of `Number(0)` in order, we can see that the token stream was successfully parsed:
1. `True` - The grammar function returned success.
2. `2` - The grammar function left the token index at position 2 (at `"EOF"`).
3. `['Number1', 'SUB', 'NUMBER:1.2']` - The subtree returned was a parse tree leaf consisting of the 2nd rule to replace `<Number>` that consists of two tokens: `SUB` and `NUMBER:1.2`.
Great! This is exactly what we would expect from our parser. 

The other grammar function that is a base case is `Name`:

In [4]:
def Name(token_index):
    '''<Name> ->
        IDENT
        | SUB IDENT
        | ADD IDENT
    '''
    if is_ident(tokens[token_index]):
        subtree = ["Name0", tokens[token_index]]
        return [True, token_index + 1, subtree]
    
    if "SUB" == tokens[token_index]:
        if is_ident(tokens[token_index + 1]):
            subtree = ["Name1", tokens[token_index], tokens[token_index + 1]]
            return [True, token_index + 2, subtree]
        
    if "ADD" == tokens[token_index]:
        if is_ident(tokens[token_index + 1]):
            subtree = ["Name2", tokens[token_index], tokens[token_index + 1]]
            return [True, token_index + 2, subtree]
        
    return [False, token_index, []]

Now let's build up from here with a grammar function that is not a base case.
## Another Layer: Value

`<Value>` is where we choose between using `<Name>` or `<Number>` in arithematic expressions. Luckily for us, we already have the `Name` and `Number` grammar functions to do the heavy lifting; all that we need to do is to call those functions and see which, if any, succeeds:

In [5]:
def Value(token_index):
    '''
    <Value> ->
        <Name>
        | <Number>
    '''
    (success, returned_index, returned_subtree) = Name(token_index)
    if success:
        return [True, returned_index, ["Value0", returned_subtree]]
    
    #<number>
    (success, returned_index, returned_subtree) = Number(token_index)
    if success:
        return [True, returned_index, ["Value1", returned_subtree]]
    
    return [False, token_index, []]

Running `Value` on the previous `token` list results in:

In [6]:
Value(0)

[True, 2, ['Value1', ['Number1', 'SUB', 'NUMBER:1.2']]]

## Part of Factor
`<Factor>` is one of Quirk's more interesting grammar rules as it contains 5 replacement rules (see [the description of Factor](#factors) above). We will focus on the last two of those replacement rules in this example:

In [7]:
def Factor(token_index):
    '''
    <Factor> ->
        <SubExpression>
        | <SubExpression> EXP <Factor>
        | <FunctionCall>
        | <Value> EXP <Factor>
        | <Value>
    '''
    #Factor0, Factor1, and Factor2 are ommitted for this example are in PartialParser.py
    
    #<Value> EXP <Factor>
    (success, returned_index, returned_subtree) = Value(token_index)
    if success:
        subtree = ["Factor3", returned_subtree]
        if "EXP" == tokens[returned_index]:
            subtree.append(tokens[returned_index])
            (success, returned_index, returned_subtree) = Factor(returned_index + 1)
            if success:
                subtree.append(returned_subtree)
                return [True, returned_index, subtree]

    #<Value>
    (success, returned_index, returned_subtree) = Value(token_index)
    if success:
        return [True, returned_index, ["Factor4", returned_subtree]]
    
    return [False, token_index, []]

This implmentation of `Factor` features three sections: `"Factor3"`, `"Factor4"`, and the failure case. The second section, `"Factor4"` simply calls `Value` and examples the results. The first section, `"Factor3"` is more interesting as it consists of `<Value> EXP <Factor>`. Because the `"Factor3"` replacement rule has three parts, it's implementation requires three conditionals: one to check for each part of the replacement rule. If any fail, `"Factor3"` does not parse. If all succeed, then `"Factor3"` succeeds. Note the mixed use of nonterminals and terminals in the replacement rule.

Let's try it out:

In [8]:
tokens = ["SUB", "IDENT:x", "EXP", "NUMBER:4", "EXP", "IDENT:x", "EOF"]
Factor(0)

[True,
 6,
 ['Factor3',
  ['Value0', ['Name1', 'SUB', 'IDENT:x']],
  'EXP',
  ['Factor3',
   ['Value1', ['Number0', 'NUMBER:4']],
   'EXP',
   ['Factor4', ['Value0', ['Name0', 'IDENT:x']]]]]]

Success! You should carefully inspect the output and see if it matches your idea of what the parse tree should look like for fragment of Quirk code starting with `Factor`:
```
-x ^ 4 ^ x
```
Lets intentionally try to break our parser by putting two `"EXP"` tokens in a row at indexes 2 and 3:

In [9]:
tokens = ["SUB", "IDENT:X", "EXP", "EXP", "NUMBER:4", "EXP", "IDENT:X", "EOF"]
Factor(0)

[True, 2, ['Factor4', ['Value0', ['Name1', 'SUB', 'IDENT:X']]]]

You may be saying "Hey, wait! Wasn't that suppposed to fail? The first value is `True`, not `False`!" You would be correct as there are still unconsumed tokens that are not `"EOF"` (the return values left the token stream at index 2). In this case, we are only using a small amout of the grammar. Your parser should check for unused tokens that are not `"EOF"` after you parse the entire program with the `Program` grammar function over the entire token stream. You should then check to see if the 2nd value in the list of return values indexes to `"EOF"` in the `tokens` list. If it does, you successfully (please check for bugs!) parsed the token stream and generated a parse tree for the interpreter to use.